In [6]:
import os
import pandas as pd
from words_list import include_list

In [2]:
folder_path = "accessible-review"
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".json")]

dfs = []
for file_path in file_paths:
    print(f"----- Reading {file_path}")
    df = pd.read_json(file_path, lines=True)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

----- Reading accessible-review/filtered_review-Colorado.json
----- Reading accessible-review/filtered_review-Nevada.json
----- Reading accessible-review/filtered_review-Missouri.json
----- Reading accessible-review/filtered_review-Arkansas.json
----- Reading accessible-review/filtered_review-Nebraska.json
----- Reading accessible-review/filtered_review-Indiana.json
----- Reading accessible-review/filtered_review-South_Carolina.json
----- Reading accessible-review/filtered_review-Kentucky.json
----- Reading accessible-review/filtered_review-Arizona.json
----- Reading accessible-review/filtered_review-Michigan.json
----- Reading accessible-review/filtered_review-Georgia.json
----- Reading accessible-review/filtered_review-Rhode_Island.json
----- Reading accessible-review/filtered_review-Oregon.json
----- Reading accessible-review/filtered_review-Delaware.json
----- Reading accessible-review/filtered_review-Washington.json
----- Reading accessible-review/filtered_review-Iowa.json
----- R

In [5]:
print(len(df))
df.head(5)

1013771


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.020905e+20,Veronica Driscoll,1552149518335,5.0,I moved my Mother to Peregrine ..Now Renew... ...,None,None,0x876c8c3818f4c2ad:0x619718e9346a49ae
1,1.122707e+20,Angela Schnaubelt,1597429653933,5.0,Greg makes the information accessible and not ...,None,"{'time': 1599066849245, 'text': 'Thanks Angela!'}",0x876c956845c25ec9:0x969fd2dfadfe606b
2,1.010503e+20,Betty Bechtel,1613106214449,5.0,We contracted with Doyle & Associates to list ...,None,"{'time': 1613428233820, 'text': 'I am always s...",0x87471d22b442a4cb:0x4400898f020fe6b5
3,1.023195e+20,Bradford Lester,1613583017163,5.0,"Ryan and his partner were on time, pleasant an...",None,None,0x876c80058ddf538f:0xd55f4745bb402a7b
4,1.015799e+20,Patrick Sillix,1614110071958,5.0,"Very polite, professional staff with punctual,...",None,"{'time': 1614536662342, 'text': 'Hi Patrick, ...",0x87471db2ccfb0ee9:0xa58d393033dc9e89


In [8]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


def split_comment(text):
    """Split the text into sentences."""
    text = text.strip()
    text = text.replace("\n", ". ")
    text = text.replace("\r", ". ")
    text = text.replace("\r\n", ". ")
    text = text.replace("----", ".")
    text = text.replace("...", ".")
    text = text.replace("(Translated by Google)", "")
    text_list = re.split(r'(?<=[.!?])\s+', text)
    text_list = [text for text in text_list if text.strip()]
    text_list = [text.strip() for text in text_list]
    return text_list


def process_comment(text):
    """Process the comment text and ensure that each is related to parking."""
    text_list = split_comment(text)
    word_list = include_list
    text_list = [text for text in text_list if any(word in text.lower() for word in word_list)]
    filtered_text = ' '.join(map(str, text_list))
    filtered_text = filtered_text.strip()
    return filtered_text   

[nltk_data] Downloading package punkt to /Users/lingyaoli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lingyaoli/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
sample_df = df.sample(3000)
sample_df['target_text'] = sample_df['text'].apply(process_comment)
sample_df.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id,target_text
354594,1.075345e+20,Alexander Corley,1592074293552,1.0,Do not stay here. They will rob you upon moveo...,None,None,0x80dc7eafb7c498d3:0xf8f9a037c99de110,They will rob you blind and send you the bill.
699779,1.182790e+20,David Penland,1546566813956,5.0,Love the place. Im disabled so Im always dropp...,None,"{'time': 1558987806128, 'text': 'THAT'S what w...",0x864a2fceede69ec3:0x8fea08ef9b4c3ab8,Im disabled so Im always dropping batteries an...
901929,1.116262e+20,M. Christine VanDusen,1522792036758,5.0,Very friendly staff. Verg accommidating. Cha...,None,"{'time': 1522854471187, 'text': 'Thank you so ...",0x88732321985c8869:0xb34a5d329ddfe203,Changed our room to handicappedat the last min.
743132,1.061040e+20,Bobbie Harrison,1555953988439,5.0,Truly one stop shopping. The could use some mo...,None,None,0x865c6046cef9e2c3:0xf91ef983ff7fcdd7,"The could use some more handicapped carts, and..."
876599,1.052639e+20,Cara Linebaugh,1599942114986,3.0,It was a very clean and accessible establishment,None,None,0x885a8d7246131e2f:0x822c9627248ad4a5,It was a very clean and accessible establishment


In [16]:
samples = sample_df[['user_id', 'gmap_id', 'text', 'target_text']]
samples = samples.drop_duplicates(subset='target_text')
samples = samples.dropna(subset=['user_id'])
print(len(samples))
samples.to_excel("samples.xlsx", index=False)

2842
